In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
from tqdm import tqdm
import cv2
import os

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
img_width = 128
img_height = 128
images=[]
classes=[]
for i in range(654):
    for x in os.listdir("train"):
        if i+1<=len(os.listdir('train/'+x)) :
            img = cv2.imread('train/'+x+'/'+os.listdir(os.path.join("train",x))[i])
            resize_image = cv2.resize(img, (img_width,img_height))
            images.append(resize_image)
            classes.append(x)

In [3]:
len(images)

4750

In [4]:
len(classes)

4750

In [5]:
print(classes[3800:4000])

['Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Scentless Mayweed', 'Loose Silky-bent', 'Fat Hen', 'Sugar beet', 'Small-flowered Cranesbill', 'Charlock', 'Common Chickweed', 'Sce

In [6]:
classes[1]

'Common wheat'

In [7]:
y = np.array(pd.get_dummies(classes,dtype="float32"))

In [8]:
def init_weight(shape):
    w=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return w

In [9]:
def init_biases(shape):
    b=tf.Variable(tf.constant(0.1,shape=shape))
    return b

In [10]:
def conv2d(X1,w):
    return tf.nn.conv2d(X1,w,strides=[1,1,1,1],padding='SAME')

In [11]:
def max_pool(X1):
    return tf.nn.max_pool(X1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [12]:
def convolution_layer(X1,shape):
    w=init_weight(shape)
    b=init_biases([shape[3]])
    return tf.nn.relu(tf.layers.batch_normalization(conv2d(X1,w)+b))

In [13]:
def fully_connected_layer(X1,size):
    feature=int(X1.get_shape()[1])
    w=init_weight([feature,size])
    b=init_biases([size])
    return tf.layers.batch_normalization((tf.matmul(X1,w)+b))

In [14]:
X1=tf.placeholder(tf.float32,shape=[None,128,128,3])
y_true=tf.placeholder(tf.float32,shape=[None,12])

In [15]:
conv_1=convolution_layer(X1,shape=[3,3,3,32])
conv_2=convolution_layer(conv_1,shape=[3,3,32,32])
conv_3=convolution_layer(conv_2,shape=[1,1,32,64])
maxpool1=max_pool(conv_3)

In [16]:
conv_4=convolution_layer(maxpool1,shape=[3,3,64,64])
conv_5=convolution_layer(conv_4,shape=[3,3,64,64])
conv_6=convolution_layer(conv_5,shape=[1,1,64,128])
maxpool2=max_pool(conv_6)

In [17]:
conv_7=convolution_layer(maxpool2,shape=[3,3,128,128])
conv_8=convolution_layer(conv_7,shape=[3,3,128,128])
conv_9=convolution_layer(conv_8,shape=[1,1,128,256])
maxpool3=max_pool(conv_9)

In [18]:
#conv_10=convolution_layer(maxpool3,shape=[3,3,256,256])
#conv_8=convolution_layer(conv_7,shape=[3,3,256,256])
#conv_9=convolution_layer(conv_8,shape=[1,1,256,512])
#maxpool4=max_pool(conv_9)

In [19]:
depth_flat=tf.reshape(maxpool3,[-1,256*256])
fcn1=tf.nn.relu(fully_connected_layer(depth_flat,2048))
fcn2=tf.nn.relu(fully_connected_layer(fcn1,512))
y_pred=(fully_connected_layer(fcn2,12))

In [20]:
with tf.name_scope('Loss'):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
with tf.name_scope('Adam'):
    train_op=tf.train.AdamOptimizer().minimize(loss_op)    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [21]:
with tf.name_scope('Accuracy'):
    prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

In [22]:
images_train=images[:4000]
y_train=y[:4200]
images_test=images[4201:4300]
y_test=y[4201:4300]

In [23]:
tf.summary.scalar("loss", loss_op)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()
saver=tf.train.Saver()

In [24]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./output1', sess.graph)
    steps=125
    sess.run(tf.global_variables_initializer())
    for j in range(60):        
        for i in range(steps):         
            batch_x=(images_train[32*i:32*(i+1)])
            batch_y=(y_train[32*i:32*(i+1)])            
            print(sess.run([train_op,loss_op,accuracy],feed_dict={X1:batch_x,y_true:batch_y}))
            _, c, summary = sess.run([train_op, loss_op, merged_summary_op],feed_dict={X1: batch_x, y_true: batch_y})
            summary_writer.add_summary(summary,i)
        print('epoch',j,' loss,accuracy:',sess.run([loss_op,accuracy],feed_dict={X1:images_test,y_true:y_test}))    

[None, 2234.0017, 0.09375]
[None, 6599.63, 0.125]
[None, 3367.1995, 0.0625]
[None, 1171.7695, 0.15625]
[None, 895.3958, 0.09375]
[None, 413.997, 0.0625]
[None, 274.31586, 0.0625]
[None, 220.69737, 0.0625]
[None, 126.43602, 0.09375]
[None, 81.54825, 0.09375]
[None, 51.37966, 0.09375]
[None, 33.882767, 0.09375]
[None, 36.42022, 0.125]
[None, 32.524117, 0.09375]
[None, 28.879776, 0.09375]
[None, 18.780973, 0.03125]
[None, 11.353842, 0.125]
[None, 11.337959, 0.09375]
[None, 8.228246, 0.09375]
[None, 9.021091, 0.03125]
[None, 7.6372156, 0.125]
[None, 9.266724, 0.09375]
[None, 6.5922155, 0.09375]
[None, 6.4150925, 0.09375]
[None, 6.0424833, 0.09375]
[None, 4.4714537, 0.0625]
[None, 4.0315666, 0.125]
[None, 4.033547, 0.09375]
[None, 4.340353, 0.0625]
[None, 3.0511644, 0.125]
[None, 3.406498, 0.1875]
[None, 3.4329767, 0.03125]
[None, 3.360198, 0.15625]
[None, 3.1370437, 0.125]
[None, 3.2717009, 0.03125]
[None, 2.9206767, 0.15625]
[None, 2.7129226, 0.09375]
[None, 2.8333135, 0.0625]
[None, 2.79

[None, 1.3271672, 0.5625]
[None, 1.2493246, 0.625]
[None, 1.3753921, 0.4375]
[None, 1.2915854, 0.59375]
[None, 1.6723568, 0.5]
[None, 1.3787522, 0.53125]
[None, 1.5078454, 0.4375]
[None, 1.6779764, 0.5]
[None, 0.99157655, 0.53125]
[None, 1.2173412, 0.59375]
[None, 1.1442933, 0.5625]
[None, 1.9264874, 0.4375]
[None, 1.2486949, 0.53125]
[None, 1.1734091, 0.5625]
[None, 1.265324, 0.625]
[None, 1.2837412, 0.5625]
[None, 1.0040727, 0.625]
[None, 0.93170184, 0.65625]
[None, 1.1028371, 0.59375]
[None, 1.2419633, 0.40625]
[None, 1.0194612, 0.65625]
[None, 1.4306862, 0.625]
[None, 2.1346815, 0.40625]
[None, 1.3747993, 0.46875]
[None, 1.062339, 0.6875]
[None, 1.0298886, 0.5625]
[None, 1.6293285, 0.5625]
[None, 1.0710065, 0.65625]
[None, 1.3933891, 0.625]
[None, 1.1053137, 0.5625]
[None, 1.3577943, 0.625]
[None, 1.6145973, 0.53125]
[None, 1.1253624, 0.59375]
[None, 1.861592, 0.5]
[None, 1.3984668, 0.6875]
[None, 1.8359233, 0.46875]
[None, 1.5997899, 0.4375]
[None, 1.2640493, 0.625]
[None, 2.18890

[None, 0.649792, 0.90625]
[None, 0.5790236, 0.75]
[None, 0.43079543, 0.90625]
epoch 4  loss,accuracy: [0.5262526, 0.8484849]
[None, 3.5670815, 0.46875]
[None, 2.58659, 0.46875]
[None, 2.3795714, 0.40625]
[None, 1.2613013, 0.5625]
[None, 1.0748311, 0.5625]
[None, 1.8530529, 0.5625]
[None, 0.79873556, 0.78125]
[None, 0.86568785, 0.6875]
[None, 0.92684126, 0.65625]
[None, 1.062587, 0.59375]
[None, 0.7434168, 0.75]
[None, 1.1576611, 0.625]
[None, 0.7434336, 0.71875]
[None, 0.9700341, 0.71875]
[None, 0.72330177, 0.71875]
[None, 0.74771905, 0.5625]
[None, 0.6580182, 0.78125]
[None, 1.3229048, 0.53125]
[None, 0.9949531, 0.65625]
[None, 0.8641491, 0.6875]
[None, 0.8947409, 0.75]
[None, 0.864975, 0.6875]
[None, 0.97266036, 0.71875]
[None, 0.8888025, 0.625]
[None, 0.7686725, 0.75]
[None, 0.9106299, 0.65625]
[None, 0.9063293, 0.6875]
[None, 0.71061474, 0.75]
[None, 0.7737402, 0.78125]
[None, 0.81126964, 0.65625]
[None, 0.8197456, 0.6875]
[None, 0.6064025, 0.75]
[None, 0.8016358, 0.8125]
[None, 0.

[None, 0.8021516, 0.6875]
[None, 0.42995358, 0.84375]
[None, 0.29426524, 0.90625]
[None, 0.44715467, 0.78125]
[None, 0.5727964, 0.84375]
[None, 0.44396636, 0.8125]
[None, 0.5365547, 0.75]
[None, 0.5941489, 0.84375]
[None, 0.52911454, 0.84375]
[None, 0.5217414, 0.6875]
[None, 0.6129161, 0.75]
[None, 0.41340387, 0.875]
[None, 0.66988546, 0.71875]
[None, 0.66408527, 0.78125]
[None, 0.435777, 0.84375]
[None, 0.9154879, 0.78125]
[None, 0.45520556, 0.875]
[None, 0.6600003, 0.8125]
[None, 0.5756198, 0.875]
[None, 0.4023679, 0.84375]
[None, 0.38751185, 0.84375]
[None, 0.3184713, 0.875]
[None, 0.5451833, 0.84375]
[None, 0.48128358, 0.84375]
[None, 0.32075208, 0.90625]
[None, 0.19329935, 0.9375]
[None, 0.39566958, 0.84375]
[None, 0.38749343, 0.90625]
[None, 0.37419906, 0.8125]
[None, 0.66615486, 0.8125]
[None, 0.34699208, 0.875]
[None, 0.38271716, 0.84375]
[None, 0.5071397, 0.84375]
[None, 0.4226988, 0.84375]
[None, 0.43012694, 0.90625]
[None, 0.32373363, 0.90625]
[None, 0.34131515, 0.875]
[None

[None, 0.19864705, 0.875]
[None, 0.6531537, 0.71875]
[None, 0.105376855, 0.96875]
[None, 0.7515533, 0.84375]
[None, 0.3223024, 0.875]
[None, 0.39088857, 0.78125]
[None, 0.47881797, 0.84375]
[None, 0.5533719, 0.9375]
[None, 0.33470434, 0.875]
[None, 0.55453897, 0.84375]
[None, 0.16508603, 0.90625]
[None, 0.08108679, 0.96875]
[None, 0.46453053, 0.8125]
[None, 0.14027461, 0.9375]
[None, 0.51064193, 0.84375]
[None, 0.41357237, 0.9375]
[None, 0.5652944, 0.84375]
[None, 0.27014744, 0.9375]
[None, 0.32347617, 0.90625]
[None, 0.3743338, 0.84375]
[None, 0.39930296, 0.90625]
[None, 0.2677046, 0.90625]
epoch 9  loss,accuracy: [1.4464401, 0.7676768]
[None, 1.2880927, 0.6875]
[None, 0.8863913, 0.71875]
[None, 1.0736121, 0.71875]
[None, 0.67531335, 0.84375]
[None, 0.38269413, 0.875]
[None, 0.7469768, 0.75]
[None, 0.39229634, 0.875]
[None, 0.5111997, 0.8125]
[None, 0.29757035, 0.84375]
[None, 0.392721, 0.84375]
[None, 0.5986803, 0.75]
[None, 0.9133707, 0.78125]
[None, 0.18313682, 0.9375]
[None, 0.502

...

[None, 0.28099912, 0.9375]
[None, 0.13551086, 0.9375]
[None, 0.035442658, 0.96875]
[None, 0.005734287, 1.0]
[None, 0.036498707, 0.96875]
[None, 0.7277564, 0.8125]
[None, 0.13236155, 0.9375]
[None, 0.08135917, 0.96875]
[None, 0.06640692, 0.96875]
[None, 0.07489203, 0.96875]
[None, 0.011865025, 1.0]
[None, 0.1282611, 0.96875]
[None, 0.0775743, 0.96875]
[None, 0.03016996, 1.0]
[None, 0.030384835, 1.0]
[None, 0.47551093, 0.84375]
[None, 0.17092562, 0.96875]
[None, 0.026915476, 0.96875]
[None, 0.38434806, 0.90625]
[None, 0.34794202, 0.90625]
[None, 0.07116653, 0.96875]
[None, 0.09377465, 0.9375]
[None, 0.1935561, 0.9375]
[None, 0.042774234, 0.96875]
[None, 0.111297585, 0.96875]
[None, 0.26595217, 0.9375]
[None, 0.019681688, 1.0]
[None, 0.06505861, 1.0]
[None, 0.20771748, 0.9375]
[None, 0.14699532, 0.9375]
[None, 0.022546658, 1.0]
[None, 0.30150405, 0.9375]
[None, 0.015718555, 1.0]
[None, 0.05371974, 0.96875]
[None, 0.29108107, 0.875]
[None, 0.16153663, 0.96875]
[None, 0.28961974, 0.90625]
[

[None, 0.0046248054, 1.0]
[None, 0.08668578, 0.96875]
[None, 0.06765865, 0.96875]
[None, 0.50694406, 0.875]
[None, 0.08231143, 0.96875]
[None, 0.7461994, 0.90625]
[None, 0.028582674, 0.96875]
[None, 0.5758647, 0.875]
[None, 0.0020947491, 1.0]
[None, 0.04668495, 1.0]
[None, 0.25697637, 0.90625]
[None, 0.15186054, 0.96875]
[None, 0.1393852, 0.9375]
[None, 0.07695382, 0.96875]
[None, 0.32519946, 0.9375]
[None, 0.0016910606, 1.0]
epoch 21  loss,accuracy: [0.51895505, 0.90909094]
[None, 0.3674831, 0.875]
[None, 0.16720495, 0.9375]
[None, 0.4934411, 0.875]
[None, 0.6085311, 0.84375]
[None, 0.012556124, 1.0]
[None, 0.52065575, 0.875]
[None, 0.01595129, 1.0]
[None, 0.062144555, 0.96875]
[None, 0.10700798, 0.96875]
[None, 0.058752347, 0.96875]
[None, 0.16602771, 0.90625]
[None, 0.08831805, 0.96875]
[None, 0.1553098, 0.90625]
[None, 0.17259724, 0.9375]
[None, 0.007263424, 1.0]
[None, 0.16851823, 0.9375]
[None, 0.0720372, 0.96875]
[None, 0.139933, 0.9375]
[None, 0.059077628, 1.0]
[None, 0.0779899

[None, 0.07110044, 0.96875]
[None, 0.10795334, 0.9375]
[None, 0.04511611, 0.96875]
[None, 0.051698014, 0.96875]
[None, 0.0067590997, 1.0]
[None, 0.077686206, 0.9375]
[None, 0.018916976, 1.0]
[None, 0.056424342, 0.96875]
[None, 0.39177886, 0.90625]
[None, 0.010369426, 1.0]
[None, 0.12707804, 0.9375]
[None, 0.045905255, 0.96875]
[None, 0.08119614, 0.96875]
[None, 0.377624, 0.9375]
[None, 0.0039526233, 1.0]
[None, 0.0027770633, 1.0]
[None, 0.18617699, 0.96875]
[None, 0.05146554, 0.96875]
[None, 0.06855152, 0.96875]
[None, 0.39861822, 0.9375]
[None, 0.07103883, 0.9375]
[None, 0.06491325, 0.96875]
[None, 0.00906398, 1.0]
[None, 0.080128044, 0.9375]
[None, 0.28489295, 0.9375]
[None, 0.019245591, 1.0]
[None, 0.011900342, 1.0]
[None, 0.095792495, 0.9375]
[None, 0.0014568332, 1.0]
[None, 0.27278697, 0.9375]
[None, 0.20405684, 0.9375]
[None, 0.27407372, 0.9375]
[None, 0.01699703, 1.0]
[None, 0.060001593, 0.96875]
[None, 0.0012010296, 1.0]
[None, 0.11061011, 0.9375]
[None, 0.25374064, 0.9375]
[No

[None, 0.13643639, 0.9375]
[None, 0.19582692, 0.90625]
[None, 0.17662424, 0.9375]
[None, 0.0061862725, 1.0]
[None, 0.066391386, 0.96875]
[None, 0.024097694, 1.0]
[None, 0.28492162, 0.9375]
[None, 0.017043218, 1.0]
[None, 0.079912074, 0.96875]
[None, 0.09347086, 0.9375]
[None, 0.28146148, 0.96875]
[None, 0.08444382, 0.9375]
[None, 0.08332319, 0.9375]
[None, 0.28226015, 0.9375]
[None, 0.037661254, 1.0]
[None, 0.0017834608, 1.0]
[None, 0.00062682736, 1.0]
[None, 0.22832352, 0.9375]
[None, 0.08382949, 0.9375]
[None, 0.15930182, 0.875]
[None, 0.14869532, 0.96875]
[None, 0.14227587, 0.9375]
[None, 0.1488172, 0.9375]
[None, 0.026179638, 1.0]
[None, 0.0235067, 1.0]
[None, 0.044795863, 0.96875]
[None, 0.3387063, 0.9375]
[None, 0.06030301, 0.96875]
[None, 8.098278e-05, 1.0]
[None, 8.603048e-05, 1.0]
[None, 0.010827484, 1.0]
[None, 0.09025206, 0.96875]
[None, 0.056763187, 0.96875]
[None, 0.00667028, 1.0]
epoch 26  loss,accuracy: [0.51836795, 0.90909094]
[None, 1.0586952, 0.875]
[None, 0.027579533

KeyboardInterrupt: 